For this example we will use the [YOLOv3](https://pjreddie.com/darknet/yolo/), a convolutional neural network for real-time object detection, wich is build on top of [Darknet](https://pjreddie.com/darknet/): an open source neural network framework written in C and CUDA. YOLOv3 can run on both CPUs and GPUs, and its source code is transparent and "easy" to modify for specific needs (e.g. data I/O format).

Download Darknet to local folder

In [2]:
! git clone https://github.com/pjreddie/darknet
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5901, done.
remote: Total 5901 (delta 0), reused 0 (delta 0), pack-reused 5901
Receiving objects: 100% (5901/5901), 6.17 MiB | 2.96 MiB/s, done.
Resolving deltas: 100% (3923/3923), done.


Change configuration files to run on GPUs (by default YOLOv3 runs on CPUs)

In [7]:

! sed -i 

/Users/paolofer/work/gfw-problem/darknet


In [6]:
! pwd

/Users/paolofer/work/gfw-problem
